**1. Upload the video file to data folder**


In [26]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [27]:
# Відкриваємо файли
cap = cv2.VideoCapture("data/Match.mp4")
template = cv2.imread("data/template.png", 0)
h, w = template.shape[:2]

In [28]:
# Output data
output_size = (1080, 1920)
fourcc = cv2.VideoWriter.fourcc(*"mp4v")
out = cv2.VideoWriter("data/output2.mp4", fourcc, 30.0, output_size)

In [29]:
# Source points from video frame
pts_src = np.array([[210, 60], [945, 60], [1113, 1240], [114, 1240]], dtype=np.float32)

# Corresponding points on the "straightened" image
pts_dst = np.array([[0, 0], [1080, 0], [1080, 1920], [0, 1920]], dtype=np.float32)

# Homography matrix for perspective correction
H, _ = cv2.findHomography(pts_src, pts_dst)

In [30]:
tracker = None
detect_threshold = 0.83  # For template matching
template_verify_threshold = 0.65  # For checking if tracking is still on the ball
frame_count = 0

In [31]:
last_valid_positions = []  # Store last few valid positions to detect jumps
max_positions = 15  # Number of positions to keep in history
max_jump_distance = 50  # Maximum allowed jump distance between frames

⚠️ **To Fix:** Трекінг в цілому працює, але коли гравець різко б'є по м'ячу, то трекінг переключається на гравця і застрягає на ньому


In [32]:
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Apply perspective transformation to the frame
        warped = cv2.warpPerspective(frame, H, (1080, 1920))
        gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)  # у сірий

        # Search for the ball
        if tracker is None:
            # If tracking is not active - search for the ball using template matching
            res = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
            print(f"[Detection] Accuracy: {max_val:.3f}")

            # If the match value exceeds the threshold - object found
            if max_val >= detect_threshold:
                print(f"[Detection] Match: {max_val:.3f}")
                top_left = max_loc
                bbox = (top_left[0], top_left[1], w, h)

                # Initialize tracker
                tracker = cv2.legacy.TrackerCSRT_create()
                tracker.init(warped, bbox)

                bottom_right = (top_left[0] + w, top_left[1] + h)

                cv2.rectangle(warped, top_left, bottom_right, (255, 0, 0), 2)
                cv2.putText(
                    warped,
                    f"Detected {max_val:.3f}",
                    (top_left[0], top_left[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255, 0, 0),
                    2,
                )

                # Add position to history
                ball_center = (top_left[0] + w // 2, top_left[1] + h // 2)
                last_valid_positions.append(ball_center)
                if len(last_valid_positions) > max_positions:
                    last_valid_positions.pop(0)
        else:
            frame_count += 1
            success, bbox = tracker.update(warped)
            print(f"[Tracking] Tracker active, bbox: {bbox}")

            if success:
                x, y, bw, bh = [int(v) for v in bbox]
                current_center = (x + bw // 2, y + bh // 2)

                # Check if ball jumped too far - this indicates tracker drift
                if len(last_valid_positions) > 0:
                    last_pos = last_valid_positions[-1]
                    distance = np.sqrt(
                        (current_center[0] - last_pos[0]) ** 2
                        + (current_center[1] - last_pos[1]) ** 2
                    )
                    if distance > max_jump_distance:
                        print(
                            f"[Warning] Ball jumped too far ({distance:.1f} pixels), likely tracker drift"
                        )
                        tracker = None
                        continue

                # 1. Template verification check
                if frame_count % 20 == 0:  # Check template every 20 frames
                    roi = gray[y : y + bh, x : x + bw]
                    if roi.shape[0] >= h and roi.shape[1] >= w:
                        roi_resized = cv2.resize(roi, (w, h))
                        res = cv2.matchTemplate(roi_resized, template, cv2.TM_CCOEFF_NORMED)
                        _, match_val, _, _ = cv2.minMaxLoc(res)
                        print(f"[Tracking] Match to template: {match_val:.3f}")

                        if match_val < template_verify_threshold:
                            print("[Warning] Tracker drifted from template - resetting")
                            tracker = None
                            continue

                # 2. Circularity check
                roi = gray[
                    max(0, y) : min(gray.shape[0], y + bh),
                    max(0, x) : min(gray.shape[1], x + bw),
                ]
                if roi.size > 0:  # Make sure ROI is valid
                    # Apply thresholding to isolate the ball from background
                    _, roi_thresh = cv2.threshold(roi, 170, 255, cv2.THRESH_BINARY)

                    # Apply morphological operations to clean up the mask
                    kernel = np.ones((3, 3), np.uint8)
                    roi_thresh = cv2.morphologyEx(roi_thresh, cv2.MORPH_OPEN, kernel)
                    roi_thresh = cv2.morphologyEx(roi_thresh, cv2.MORPH_CLOSE, kernel)

                    contours, _ = cv2.findContours(
                        roi_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
                    )
                    valid_contour = False

                    for cnt in contours:
                        if len(cnt) > 5:  # Need at least 5 points to fit an ellipse
                            area = cv2.contourArea(cnt)
                            if area > 30:  # Ignore tiny contours
                                perimeter = cv2.arcLength(cnt, True)
                                circularity = 4 * np.pi * area / (perimeter**2 + 1e-5)
                                print(f"[Circularity] Value: {circularity:.3f}")

                                if circularity > 0.7:  # Closer to 1 = circle
                                    valid_contour = True
                                    # Draw the circular contour
                                    cv2.drawContours(
                                        warped,
                                        [cnt + np.array([max(0, x), max(0, y)])],
                                        -1,
                                        (0, 255, 0),
                                        2,
                                    )

                    if (
                        not valid_contour and frame_count % 5 == 0
                    ):  # Only check every 5 frames to avoid flickering
                        print("[Warning] No circular object in ROI - might have drifted")
                        # We'll let template matching confirm this in next verification

                # If we've passed all checks, update history
                last_valid_positions.append(current_center)
                if len(last_valid_positions) > max_positions:
                    last_valid_positions.pop(0)

                # Draw rectangle around the tracked ball
                cv2.rectangle(warped, (x, y), (x + bw, y + bh), (255, 0, 0), 2)
                cv2.putText(
                    warped,
                    f"Tracked",
                    (x + 10, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255, 0, 0),
                    2,
                )

                # Draw trajectory
                if len(last_valid_positions) > 1:
                    for i in range(1, len(last_valid_positions)):
                        cv2.line(
                            warped,
                            last_valid_positions[i - 1],
                            last_valid_positions[i],
                            (0, 0, 255),
                            2,
                        )
            else:
                print("[Info] Tracker lost - retrying.")
                tracker = None

        # Записуємо оброблений кадр у відео
        out.write(warped)

        # зменшую розмір кадру в 2 рази для зручного перегляду
        scaled_warped = cv2.resize(warped, (0, 0), fx=0.5, fy=0.5)
        cv2.imshow("Warped View", scaled_warped)

        # вихід по клавіші Esc
        if cv2.waitKey(30) & 0xFF == 27:
            break
finally:
    cap.release()
    out.release()
    cv2.destroyAllWindows()

[Detection] Accuracy: 0.768
[Detection] Accuracy: 0.770
[Detection] Accuracy: 0.774
[Detection] Accuracy: 0.782
[Detection] Accuracy: 0.761
[Detection] Accuracy: 0.763
[Detection] Accuracy: 0.771
[Detection] Accuracy: 0.769
[Detection] Accuracy: 0.780
[Detection] Accuracy: 0.773
[Detection] Accuracy: 0.790
[Detection] Accuracy: 0.772
[Detection] Accuracy: 0.783
[Detection] Accuracy: 0.791
[Detection] Accuracy: 0.785
[Detection] Accuracy: 0.777
[Detection] Accuracy: 0.773
[Detection] Accuracy: 0.782
[Detection] Accuracy: 0.807
[Detection] Accuracy: 0.796
[Detection] Accuracy: 0.784
[Detection] Accuracy: 0.774
[Detection] Accuracy: 0.776
[Detection] Accuracy: 0.777
[Detection] Accuracy: 0.775
[Detection] Accuracy: 0.772
[Detection] Accuracy: 0.776
[Detection] Accuracy: 0.758
[Detection] Accuracy: 0.774
[Detection] Accuracy: 0.783
[Detection] Accuracy: 0.777
[Detection] Accuracy: 0.783
[Detection] Accuracy: 0.780
[Detection] Accuracy: 0.770
[Detection] Accuracy: 0.798
[Detection] Accuracy

Gates tracking

In [ ]:
import time
from pathlib import Path
from dataclasses import dataclass
from typing import Final, Optional, Tuple, List

import cv2
import numpy as np

@dataclass(frozen=True)
class PitchCfg:
    green: Tuple[Tuple[int, int, int], Tuple[int, int, int]] = ((35, 40, 40), (85, 255, 255))
    frame_skip: int = 30
    max_corner_drift: float = 6.0

@dataclass(frozen=True)
class GoalPostCfg:
    strip_height: int = 80
    bright_hsv: Tuple[Tuple[int, int, int], Tuple[int, int, int]] = ((0, 0, 200), (180, 60, 255))
    sobel_blur: int = 21
    peak_min_ratio: float = 0.30
    min_peak_gap: int = 80
    zone_frac: float = 0.4
    max_jump: int = 30
    recheck_radius: int = 3
    recheck_min_frac: float = 0.40
    smooth_alpha: float = 0.35

LABEL_FONT: Final = cv2.FONT_HERSHEY_SIMPLEX
PITCH = PitchCfg()
POSTS = GoalPostCfg()

def order_corners(pts: np.ndarray) -> np.ndarray:
    """Return points in consistent TL-TR-BR-BL order."""
    s = pts.sum(1)
    d = np.diff(pts, axis=1)
    ordered = np.array(
        [
            pts[np.argmin(s)],
            pts[np.argmin(d)],
            pts[np.argmax(s)],
            pts[np.argmax(d)]
        ],
        dtype=np.float32,
    )
    return ordered

def detect_pitch(hsv: np.ndarray) -> Optional[np.ndarray]:
    """
    Detect the green rectangle (pitch) and return 4x2 float32 corners.
    Returns None if not found.
    """
    low, high = PITCH.green
    mask = cv2.inRange(hsv, low, high)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None
    cnt = max(cnts, key=cv2.contourArea)
    if cv2.contourArea(cnt) < 50_000:
        return None
    hull = cv2.convexHull(cnt)
    epsilon = 0.02 * cv2.arcLength(cnt, True)
    quad = cv2.approxPolyDP(hull, epsilon, True)
    return order_corners(quad.reshape(-1, 2)) if len(quad) == 4 else None

def _bright_mask(hsv: np.ndarray) -> np.ndarray:
    """Binary mask of 'bright' HSV pixels in configured range."""
    low, high = POSTS.bright_hsv
    return cv2.inRange(hsv, low, high)

def _two_outer_peaks(strip_bgr: np.ndarray) -> Optional[Tuple[int, int]]:
    """
    Return the x positions of the two bright vertical peaks (goal posts)
    or None if conditions aren't met.
    """
    hsv = cv2.cvtColor(strip_bgr, cv2.COLOR_BGR2HSV)
    mask = cv2.morphologyEx(
        _bright_mask(hsv),
        cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)),
    )
    energy = np.abs(cv2.Sobel(mask, cv2.CV_32F, 0, 1, ksize=3)).sum(axis=0)
    energy = cv2.blur(energy[:, None], (POSTS.sobel_blur, 1)).ravel()
    if energy.max() < 1:
        return None
    w = energy.size
    left = np.argmax(energy[: int(POSTS.zone_frac * w)])
    right = np.argmax(energy[int((1 - POSTS.zone_frac) * w):]) + int((1 - POSTS.zone_frac) * w)
    thresh = POSTS.peak_min_ratio * energy.max()
    if energy[left] < thresh or energy[right] < thresh or (right - left) < POSTS.min_peak_gap:
        return None
    return int(left), int(right)

def _is_confirmed(hsv_strip: np.ndarray, x: int) -> bool:
    """
    Check whether a small vertical slice near 'x' in the strip has a sufficient fraction
    of bright pixels to confirm a post.
    """
    w = hsv_strip.shape[1]
    x0, x1 = max(0, x - POSTS.recheck_radius), min(w, x + POSTS.recheck_radius + 1)
    frac = (_bright_mask(hsv_strip[:, x0:x1]) > 0).mean()
    return frac >= POSTS.recheck_min_frac

class PostTracker:
    """
    Exponential moving average tracker to keep posts stable frame-to-frame.
    """
    def __init__(self) -> None:
        self._pair: Optional[Tuple[int, int]] = None

    def update(
        self, meas: Optional[Tuple[int, int]], hsv_strip: np.ndarray
    ) -> Optional[Tuple[int, int]]:
        if meas:
            l, r = meas
            if not (_is_confirmed(hsv_strip, l) and _is_confirmed(hsv_strip, r)):
                meas = None

        if self._pair and meas:
            if max(abs(meas[0] - self._pair[0]), abs(meas[1] - self._pair[1])) > POSTS.max_jump:
                meas = None

        if meas:
            if self._pair:
                l_sm = int(POSTS.smooth_alpha * meas[0] + (1 - POSTS.smooth_alpha) * self._pair[0])
                r_sm = int(POSTS.smooth_alpha * meas[1] + (1 - POSTS.smooth_alpha) * self._pair[1])
                self._pair = (l_sm, r_sm)
            else:
                self._pair = meas
        return self._pair

def draw_black_goal(frame: np.ndarray) -> None:
    """
    A simple detector for a fixed 'black slot' goal.
    Draws a rectangle on the frame if found.
    """
    h, w = frame.shape[:2]
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (0, 0, 0), (180, 255, 40))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((7, 7), np.uint8), iterations=2)
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in cnts:
        area = cv2.contourArea(cnt)
        x, y, bw, bh = cv2.boundingRect(cnt)
        if not ((w * 0.05) * (h * 0.015) < area < (w * 0.15) * (h * 0.045)):
            continue
        ar = bw / bh
        if not (2.5 < ar < 6.0) or not (y + bh / 2 < h * 0.25) or not (w * 0.3 < x + bw / 2 < w * 0.7):
            continue
        cv2.rectangle(frame, (x, y), (x + bw, y + bh), (255, 0, 255), 2)
        break

def overlay_info(frame: np.ndarray, corners: np.ndarray) -> None:
    """
    Overlay the pitch polygon and its corner labels on the frame.
    """
    poly = corners.astype(int).reshape(-1, 2)
    cv2.polylines(frame, [poly], True, (0, 255, 0), 2)
    labels = ("TL", "TR", "BR", "BL")
    for lbl, (x, y) in zip(labels, poly):
        cv2.putText(frame, lbl, (x + 5, y - 5), LABEL_FONT, 0.5, (0, 255, 0), 1)

def process_video(src: Path, dst: Optional[Path] = None) -> List[Tuple[int, Optional[Tuple[Tuple[int, int], Tuple[int, int]]]]]:
    """
    Process the video at 'src', write annotated video to 'dst',
    and return a list of goal coordinates for each frame.
    
    Each element in the returned list is a tuple:
       (frame_index, goal_coordinates)
    
    If goals are detected in the frame, goal_coordinates is a tuple of two points:
       ((x_left, y_top), (x_right, y_bottom))
    Else it is None.
    """
    cap = cv2.VideoCapture(str(src))
    ret, frame = cap.read()

    height, width = frame.shape[:2]
    cv2.namedWindow("goals", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("goals", 960, 540)
    writer = None
    if dst:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        writer = cv2.VideoWriter(str(dst), fourcc, 30, (width, height))

    goals_coords_list: List[Tuple[int, Optional[Tuple[Tuple[int, int], Tuple[int, int]]]]] = []

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    corners = detect_pitch(hsv_frame)
    if corners is None:
        raise RuntimeError("Pitch not detected in the first frame")
    corners = corners.reshape(-1, 1, 2).astype(np.float32)

    lk_params = dict(
        winSize=(21, 21),
        maxLevel=3,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01)
    )
    prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    tracker = PostTracker()
    frame_idx = 0
    t0 = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # ── Redetect pitch every 'frame_skip' frames ──
        if frame_idx % PITCH.frame_skip == 0:
            new_detection = detect_pitch(cv2.cvtColor(frame, cv2.COLOR_BGR2HSV))
            if new_detection is not None:
                corners = new_detection.reshape(-1, 1, 2).astype(np.float32)

        new_corners, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray, corners, None, **lk_params)
        if status.sum() == 4 and np.max(np.linalg.norm(new_corners - corners, axis=2)) < PITCH.max_corner_drift:
            corners = new_corners

        poly = corners.astype(int).reshape(-1, 2)
        xL, xR = poly[:, 0].min(), poly[:, 0].max()
        yT, yB = poly[:, 1].min(), poly[:, 1].max()

        strip_top = max(int(yB - POSTS.strip_height), 0)
        strip_bottom = int(yB)
        strip_bgr = frame[strip_top:strip_bottom, xL:xR]
        strip_hsv = cv2.cvtColor(strip_bgr, cv2.COLOR_BGR2HSV)
        meas = _two_outer_peaks(strip_bgr)
        post_pair = tracker.update(meas, strip_hsv)

        goal_coords: Optional[Tuple[Tuple[int, int], Tuple[int, int]]] = None
        if post_pair:
            l, r = post_pair
            # Calculate the goal rectangle (top-left and bottom-right points)
            goal_coords = ((xL + l, strip_top), (xL + r, yB))
            cv2.rectangle(frame, (xL + l, strip_top), (xL + r, yB), (255, 0, 255), 2)

        goals_coords_list.append((frame_idx, goal_coords))

        draw_black_goal(frame)
        overlay_info(frame, corners)

        cv2.imshow("goals", frame)
        if writer:
            writer.write(frame)
        if cv2.waitKey(1) & 0xFF == 27:  # Exit on 'Esc'
            break

        prev_gray = gray
        frame_idx += 1

    cap.release()
    if writer:
        writer.release()
    cv2.destroyAllWindows()
    elapsed = time.time() - t0
    fps = frame_idx / elapsed if elapsed > 0 else 0
    print(f"[DONE] {frame_idx} frames processed at {fps:.1f} FPS")
    return goals_coords_list

goals_results = process_video(Path("data/Match.mp4"), Path("data/goals.mp4"))
for frame_idx, coords in goals_results:
    print(f"Frame {frame_idx}: {coords}")


[DONE] 10327 frames processed at 19.7 FPS
Frame 0: None
Frame 1: None
Frame 2: None
Frame 3: ((np.int64(382), 1158), (np.int64(752), np.int64(1238)))
Frame 4: ((np.int64(382), 1158), (np.int64(752), np.int64(1238)))
Frame 5: ((np.int64(382), 1158), (np.int64(749), np.int64(1238)))
Frame 6: ((np.int64(381), 1159), (np.int64(748), np.int64(1239)))
Frame 7: ((np.int64(381), 1159), (np.int64(748), np.int64(1239)))
Frame 8: ((np.int64(381), 1159), (np.int64(748), np.int64(1239)))
Frame 9: ((np.int64(381), 1159), (np.int64(748), np.int64(1239)))
Frame 10: ((np.int64(380), 1158), (np.int64(750), np.int64(1238)))
Frame 11: ((np.int64(378), 1158), (np.int64(750), np.int64(1238)))
Frame 12: ((np.int64(378), 1159), (np.int64(750), np.int64(1239)))
Frame 13: ((np.int64(378), 1159), (np.int64(750), np.int64(1239)))
Frame 14: ((np.int64(378), 1159), (np.int64(750), np.int64(1239)))
Frame 15: ((np.int64(378), 1159), (np.int64(750), np.int64(1239)))
Frame 16: ((np.int64(378), 1159), (np.int64(750), np